In [1]:
import feedparser
import bs4
import re
import pickle
from collections import defaultdict
from datetime import datetime
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import rpy2.robjects as robjects
import sklearn
import cPickle as pickle
from datetime import date

In [2]:
import nltk
# nltk.download()
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk import FreqDist

import string

/Library/Python/2.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [3]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = " " # try with a space

    for c in s:
            if c == '<' and not quote:
                tag = True
            elif c == '>' and not quote:
                tag = False
            elif (c == '"' or c == "'") and tag:
                quote = not quote
            elif not tag:
                out = out + c

    return out

## SCRAPE 

In [11]:
try:
    CFfp = feedparser.parse("http://www.crossfit.com/mt-archive2/2015_11.html")
except Exception, err:
    print err.message

html = CFfp['feed']['summary']
soup = bs4.BeautifulSoup(html, "html.parser")

links = soup.findAll("a", {"href" : re.compile("archive2/20")})
links = set([l['href'] for l in links])

In [52]:
wodDict = {}
for l in links:
    try:
        CFfp = feedparser.parse(l)
    except Exception, err:
        print err.message

    html = CFfp['feed']['summary']
    soup = bs4.BeautifulSoup(html)

    s = soup.find("div", {"class" : re.compile("date")}) 
        # First div of calss date is  Monthname YYYY 
    for i in range(len(soup.findAll("div", {"class" : re.compile("date")}))-1):
        # Convert date string to datetim
        s = s.findNext("div", {"class" : "date"}) # All Subsequent Dates are Monthname DD, YYYY
        date = datetime.strptime((str(s.getText())).strip(), "%B %d, %Y").date()

        # Extract Workout from Blogbody
        blogbody =  s.findNext("div", {"class" : "blogbody"})
        workout = re.sub(r'[\t]', '', ''.join(blogbody.findAll(text=True)))
        
        # Strip non-workout text out
        start = workout.find(date.strftime('%y%m%d'))+7
        end1 = workout.find('Post')
        end2 = workout.find('Enlarge image')
        end3 = workout.find('___')
        if end1==-1: end1=len(workout)
        if end2==-1: end2=len(workout)
        if end3==-1: end3=len(workout)
        end = min(end1, end2, end3)
        
        workout = workout[start:end].replace('\n', ' ').replace('\u', ' ')
        workout = workout.encode('ascii','ignore')
        workout = workout.rstrip()
        
        wodDict[date]=workout
        i+1

wodDict

{datetime.date(2002, 7, 29): 'Seven rounds for time of: Walking lunge, 10 steps 10 Pull ups 10 Push ups',
 datetime.date(2011, 5, 29): '"Amanda" Three rounds, 9-7- and 5 reps, for time of: Muscle-up 135 pound Squat snatch',
 datetime.date(2001, 8, 29): 'Three rounds for time of: 1000 meter Row 1/2 bodyweight Push jerk, 21 reps',
 datetime.date(2004, 6, 29): 'How many rounds in 20 minutes of: Walking lunge 50 ft. 5 Handstand push-ups',
 datetime.date(2007, 8, 21): '"Tabata Something Else" Complete 32 intervals of 20 seconds of work followed by ten seconds of rest where the first 8 intervals are pull-ups, the second 8 are push-ups, the third 8 intervals are sit-ups, and finally, the last 8 intervals are squats. There is no rest between exercises.',
 datetime.date(2002, 11, 4): 'For time: 15 Snatch 15 Knees to elbows 10 Snatch 10 Snatch 15 Knees to elbows 5 Snatch 5 Snatch 5 Snatch 15 Knees to elbows',
 datetime.date(2001, 3, 21): "Five sets of the triple, Sit-up Back extension Bench-pres

In [4]:
# pickle.dump(wodDict, open( "wodDict.p", "wb" ) )
wodDict = pickle.load( open( "wodDict.p", "rb" ) )

In [5]:
len(wodDict)/365

14

## IDentify MOVEments 

In [3]:
wodDict = pickle.load( open( "wodDict.p", "rb" ) )

In [56]:
wodText = ' '.join(wodDict.values())
print len(wodText)
wodText[0:int(round(len(wodText)*.1,0))]

686649


'Seven rounds for time of: Walking lunge, 10 steps 10 Pull ups 10 Push ups "Amanda" Three rounds, 9-7- and 5 reps, for time of: Muscle-up 135 pound Squat snatch Three rounds for time of: 1000 meter Row 1/2 bodyweight Push jerk, 21 reps How many rounds in 20 minutes of: Walking lunge 50 ft. 5 Handstand push-ups "Tabata Something Else" Complete 32 intervals of 20 seconds of work followed by ten seconds of rest where the first 8 intervals are pull-ups, the second 8 are push-ups, the third 8 intervals are sit-ups, and finally, the last 8 intervals are squats. There is no rest between exercises. For time: 15 Snatch 15 Knees to elbows 10 Snatch 10 Snatch 15 Knees to elbows 5 Snatch 5 Snatch 5 Snatch 15 Knees to elbows Five sets of the triple, Sit-up Back extension Bench-press 5-10RM Move methodically, deliberately, with precise SLOW movement.  Don\'t hurry; minimize the cardiovascular quality. This is strength training only!  Make each effort a serious challenge. Deadlift 3-3-3-3-3 reps Five

In [6]:
# Tokenize Text: break up into a list of words and punctuation... 
tokens = nltk.word_tokenize(wodText)
tokens

NameError: name 'wodText' is not defined

In [58]:
pickle.dump(tokens, open( "tokens.p", "wb" ) )

In [59]:
filtered_tokens = [w for w in tokens if not w in stopwords.words('english') and len(w)>2]

In [60]:
filtered_tokens[0:4]

['Seven', 'rounds', 'time', 'Walking']

In [61]:
# 10 most freq words
fd = FreqDist([w.lower() for w in filtered_tokens])
fd.most_common(10)

[('reps', 3270),
 ('rest', 2107),
 ('time', 2060),
 ('rounds', 2038),
 ('pull-ups', 1578),
 ('pound', 1514),
 ('day', 1330),
 ('run', 1264),
 ('meters', 1094),
 ('minutes', 1056)]

In [62]:
# Creating an NLTK text from the list of words and punctiation resulting from tokenization:
text = nltk.Text(tokens)
print type(text)
text[1000:1010]

<class 'nltk.text.Text'>


['Mikko', 'Salo', '16', 'rounds', '+', '3', 'SDHP', ',', 'Elyse', 'Umeda']

In [63]:
# sequence of words that occur together unusually often
text.collocations()


Rest Day; 400 meters; Run 400; Five rounds; Three rounds; many rounds;
Run 800; 800 meters; Handstand push-ups; Kristan Clever; body weight;
Rope climb; 135 pound; Ring dips; Box jump; Kettlebell swings; pound
Thruster; Bench press; Rebecca Voigt; CrossFit Journal


In [64]:
text.concordance('pull-ups')

Displaying 25 of 1578 matches:
st where the first 8 intervals are pull-ups , the second 8 are push-ups , the 
/3 your bodyweight 20 Thrusters 20 Pull-ups Rest three minutes . Notes : 1 . M
 and arm extension . 4 . No crappy pull-ups . Adam 's apple to bar at top , ar
xtended at bottom . 5 . The twenty pull-ups are consecutive pull-ups . You do 
he twenty pull-ups are consecutive pull-ups . You do n't come down from the ba
 50 reps Sit-ups , 50 reps Jumping pull-ups , 50 reps 50 Back extension Dips ,
 in 20 minutes of : 7 chest-to-bar pull-ups 77 double-unders 170-lb . squat cl
 snatch , 21 reps , right arm 21 L Pull-ups 40 pound Dumbbell snatch , 21 reps
l snatch , 21 reps , left arm 21 L Pull-ups Overhead Squat 5-5-5-5-5 reps `` L
 reps of : Body weight bench press Pull-ups Rest Day In front of a clock set f
ds in 20 minutes as you can of : 5 Pull-ups 10 Push-ups 15 Squats OR `` Mary '
One legged squats , alternating 15 Pull-ups Complete as many rounds as possibl
s 6 clean and jerks 9

In [65]:
# Stemming:  stripping off any affixes
# The Porter Stemmer is a good choice if you are indexing some texts 
# and want to support search using alternative forms of words

porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()

print [porter.stem(t) for t in tokens][0:100]
print [lancaster.stem(t) for t in tokens][0:100]

[u'Seven', u'round', u'for', u'time', u'of', u':', u'Walk', u'lung', u',', u'10', u'step', u'10', u'Pull', u'up', u'10', u'Push', u'up', u'``', u'Amanda', u"''", u'Three', u'round', u',', u'9-7-', u'and', u'5', u'rep', u',', u'for', u'time', u'of', u':', u'Muscle-up', u'135', u'pound', u'Squat', u'snatch', u'Three', u'round', u'for', u'time', u'of', u':', u'1000', u'meter', u'Row', u'1/2', u'bodyweight', u'Push', u'jerk', u',', u'21', u'rep', u'How', u'mani', u'round', u'in', u'20', u'minut', u'of', u':', u'Walk', u'lung', u'50', u'ft.', u'5', u'Handstand', u'push-up', u'``', u'Tabata', u'Someth', u'Els', u"''", u'Complet', u'32', u'interv', u'of', u'20', u'second', u'of', u'work', u'follow', u'by', u'ten', u'second', u'of', u'rest', u'where', u'the', u'first', u'8', u'interv', u'are', u'pull-up', u',', u'the', u'second', u'8', u'are', u'push-up']
['sev', 'round', 'for', 'tim', 'of', ':', 'walk', 'lung', ',', '10', 'step', '10', 'pul', 'up', '10', 'push', 'up', '``', 'amand', "''", 'th

In [66]:
# porter = nltk.PorterStemmer()
# grail = nltk.corpus.webtext.words('grail.txt')

class IndexedText(object):

    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width)
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width)
            print(ldisplay, rdisplay)

    def _stem(self, word):
        return self._stemmer.stem(word).lower()

text0 = IndexedText(porter, text)
text0.concordance('pull-up')

(' of rest where the first 8 intervals are', 'pull-ups , the second 8 are push-ups , t')
('ling 1/3 your bodyweight 20 Thrusters 20', 'Pull-ups Rest three minutes . Notes : 1 ')
('that Thrusters begin directly under your', 'pull-up bar . 2 . Each step of the lunge')
(' leg , and arm extension . 4 . No crappy', "pull-ups . Adam 's apple to bar at top ,")
('tely extended at bottom . 5 . The twenty', 'pull-ups are consecutive pull-ups . You ')
(' 5 . The twenty pull-ups are consecutive', "pull-ups . You do n't come down from the")
('-ups , 50 reps Sit-ups , 50 reps Jumping', 'pull-ups , 50 reps 50 Back extension Dip')
('ssible in 20 minutes of : 7 chest-to-bar', 'pull-ups 77 double-unders 170-lb . squat')
('mbbell snatch , 21 reps , right arm 21 L', 'Pull-ups 40 pound Dumbbell snatch , 21 r')
('umbbell snatch , 21 reps , left arm 21 L', 'Pull-ups Overhead Squat 5-5-5-5-5 reps `')
('or max reps of : Body weight bench press', 'Pull-ups Rest Day In front of a clock se')
('  rounds in 20 minu

In [67]:
# A further step is to make sure that the resulting form is a known word in a dictionary, a task known as lemmatization.
# The WordNet lemmatizer only removes affixes if the resulting word is in its dictionary.

# Note that pull-ups and push-ups become pull/push ups, separately


wnl = nltk.WordNetLemmatizer()
[wnl.lemmatize(t) for t in tokens]


['Seven',
 u'round',
 'for',
 'time',
 'of',
 ':',
 'Walking',
 'lunge',
 ',',
 '10',
 u'step',
 '10',
 'Pull',
 'ups',
 '10',
 'Push',
 'ups',
 '``',
 'Amanda',
 "''",
 'Three',
 u'round',
 ',',
 '9-7-',
 'and',
 '5',
 u'rep',
 ',',
 'for',
 'time',
 'of',
 ':',
 'Muscle-up',
 '135',
 'pound',
 'Squat',
 'snatch',
 'Three',
 u'round',
 'for',
 'time',
 'of',
 ':',
 '1000',
 'meter',
 'Row',
 '1/2',
 'bodyweight',
 'Push',
 'jerk',
 ',',
 '21',
 u'rep',
 'How',
 'many',
 u'round',
 'in',
 '20',
 u'minute',
 'of',
 ':',
 'Walking',
 'lunge',
 '50',
 'ft.',
 '5',
 'Handstand',
 'push-ups',
 '``',
 'Tabata',
 'Something',
 'Else',
 "''",
 'Complete',
 '32',
 u'interval',
 'of',
 '20',
 u'second',
 'of',
 'work',
 'followed',
 'by',
 'ten',
 u'second',
 'of',
 'rest',
 'where',
 'the',
 'first',
 '8',
 u'interval',
 'are',
 u'pull-up',
 ',',
 'the',
 'second',
 '8',
 'are',
 'push-ups',
 ',',
 'the',
 'third',
 '8',
 u'interval',
 'are',
 u'sit-up',
 ',',
 'and',
 'finally',
 ',',
 'the',


In [ ]:
# TFIDF

In [75]:
wodDict.keys()[0]

datetime.date(2002, 7, 29)

In [9]:
wodDict[wodDict.keys()[0]]

'Seven rounds for time of: Walking lunge, 10 steps 10 Pull ups 10 Push ups'

In [8]:
token_dict = {}
for date, wod in wodDict.iteritems():
    text = wod
    lowers = text.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[date] = no_punctuation
        

In [10]:
token_dict.keys()[0]

datetime.date(2002, 7, 29)

In [11]:
token_dict[token_dict.keys()[0]]

'seven rounds for time of walking lunge 10 steps 10 pull ups 10 push ups'

In [12]:
stemmer = nltk.PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    punctuation = re.compile(r'[-.?!,":;()|0-9]')
    word_list = [punctuation.sub("", word) for word in stems]   
    return word_list

In [13]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [123]:
#  input a new document (or existing) into the model and get a tf-idf answer back. 
# We do this by using the transform function, which will use our existing NLTK preprocessor first.
str = token_dict[token_dict.keys()[0]]
response = tfidf.transform([str])
print response

  (0, 4095)	0.341327395357
  (0, 3856)	0.157872297071
  (0, 3609)	0.396081926473
  (0, 3386)	0.42004070902
  (0, 3261)	0.166717775763
  (0, 3021)	0.323812244356
  (0, 3004)	0.415851873317
  (0, 2294)	0.352468178482
  (0, 0)	0.308935387338


In [124]:
# We can get the specific terms and their tf-idf score (but it's not straightforward):
feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print feature_names[col], ' - ', response[0, col]

walk  -  0.341327395357
time  -  0.157872297071
step  -  0.396081926473
seven  -  0.42004070902
round  -  0.166717775763
push  -  0.323812244356
pull  -  0.415851873317
lung  -  0.352468178482
  -  0.308935387338


In [15]:
# The common way of doing this is to transform the documents into tf-idf vectors, then compute the cosine similarity
# between them. Any textbook on information retrieval (IR) covers this. See esp. Introduction to Information 
# Retrieval, which is free and available online.
# Tf-idf (and similar text transformations) are implemented in the Python packages Gensim and scikit-learn. 
# In the latter package, computing cosine similarities is as easy as

token_list = token_dict.values()

tfidf = TfidfVectorizer().fit_transform(token_list)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T

In [16]:
token_list

['seven rounds for time of walking lunge 10 steps 10 pull ups 10 push ups',
 'three rounds for time of 1000 meter row 12 bodyweight push jerk 21 reps',
 'tabata something else complete 32 intervals of 20 seconds of work followed by ten seconds of rest where the first 8 intervals are pullups the second 8 are pushups the third 8 intervals are situps and finally the last 8 intervals are squats there is no rest between exercises',
 'five sets of the triple situp back extension benchpress 510rm move methodically deliberately with precise slow movement  dont hurry minimize the cardiovascular quality this is strength training only  make each effort a serious challenge',
 'deadlift 33333 reps',
 'five rounds for time of  95pound hang squat clean 21 reps run 400 meters',
 'three rounds of 20 walking lunges with dumbbells totaling 13 your bodyweight 20 thrusters 20 pullups rest three minutes  notes 1 measure your pace so that thrusters begin directly under your pullup bar  2 each step of the lun

In [126]:
print pairwise_similarity[0:5,0:5]

  (0, 3)	0.00530047327864
  (0, 2)	0.0138071297142
  (0, 1)	0.131830458322
  (0, 0)	1.0
  (1, 3)	0.00749404607238
  (1, 2)	0.0195211372204
  (1, 0)	0.131830458322
  (1, 4)	0.0446020269727
  (1, 1)	1.0
  (2, 1)	0.0195211372204
  (2, 0)	0.0138071297142
  (2, 3)	0.0660714507167
  (2, 2)	1.0
  (3, 1)	0.00749404607238
  (3, 0)	0.00530047327864
  (3, 2)	0.0660714507167
  (3, 3)	1.0
  (4, 1)	0.0446020269727
  (4, 4)	1.0


In [133]:
# Now to find the cosine distances of one document (e.g. the first in the dataset) and all of the others you 
# just need to compute the dot products of the first vector with all of the others as the tfidf vectors are 
# already row-normalized. The scipy sparse matrix API is a bit weird (not as flexible as dense N-dimensional 
# numpy arrays). To get the first vector you need to slice the matrix row-wise to get a submatrix with a single row:

# scikit-learn already provides pairwise metrics (a.k.a. kernels in machine learning parlance) 
# that work for both dense and sparse representations of vector collections. 
# In this case we need a dot product that is also known as the linear kernel:

from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(pairwise_similarity[0:1], pairwise_similarity).flatten()

# Hence to find the top 5 related documents, we can use argsort and some negative array slicing 
# (most related documents have highest cosine similarity values, hence at the end of the sorted indices array):
related_docs_indices = cosine_similarities.argsort()[:-5:-1]
print related_docs_indices
print cosine_similarities[related_docs_indices]
print "\n"
print "\n"


print token_dict.keys()[0],  token_dict[token_dict.keys()[0]]
for i in range(len(related_docs_indices)):
    print token_dict.keys()[related_docs_indices[i]],  token_dict[token_dict.keys()[related_docs_indices[i]]]

[3299  179 1809 4419]
[ 44.37819825  41.79150645  39.4866673   39.22308719]




2002-07-29 seven rounds for time of walking lunge 10 steps 10 pull ups 10 push ups
2003-10-14 15 rounds for time of  10 pullups 10 pushups 10 squats run 100 meters
2012-11-19 complete as many rounds as possible in 20 minutes of run 400 meters then three rounds of 5 pullups 10 pushups 15 squats
2010-06-11 15 rounds for time of 5 pullups 10 pushups 15 squats
2006-11-11 seven rounds for time of 135 pound clean 10 reps  handstand pushups 10 reps


In [134]:
cosine_similarities = linear_kernel(pairwise_similarity[1:2], pairwise_similarity).flatten()
related_docs_indices = cosine_similarities.argsort()[:-5:-1]


print related_docs_indices
print cosine_similarities[related_docs_indices]
print "\n"
print "\n"


print token_dict.keys()[0],  token_dict[token_dict.keys()[1]]
for i in range(len(related_docs_indices)):
    print token_dict.keys()[related_docs_indices[i]],  token_dict[token_dict.keys()[related_docs_indices[i]]]

[ 851 3010 2525  720]
[ 64.1510754   61.91046179  60.79816873  60.18557989]




2002-07-29 three rounds for time of 1000 meter row 12 bodyweight push jerk 21 reps
2014-01-09 five rounds for time of  run 400 meters 50 pound dumbbell clean  jerk 21 reps
2003-03-07 three rounds for time of 135 pound back squat 21 reps run 400 meters
2010-02-11 five rounds for time of row 500 meters 135 pound power clean 12 reps 95 pound thruster 15 reps
2004-01-28 five rounds for time of run 400 meters 95 pound hang squat clean 15 reps


## nltk with certain words weighing mroe than others?

# Gensim LDA

In [17]:
token_list

['seven rounds for time of walking lunge 10 steps 10 pull ups 10 push ups',
 'three rounds for time of 1000 meter row 12 bodyweight push jerk 21 reps',
 'tabata something else complete 32 intervals of 20 seconds of work followed by ten seconds of rest where the first 8 intervals are pullups the second 8 are pushups the third 8 intervals are situps and finally the last 8 intervals are squats there is no rest between exercises',
 'five sets of the triple situp back extension benchpress 510rm move methodically deliberately with precise slow movement  dont hurry minimize the cardiovascular quality this is strength training only  make each effort a serious challenge',
 'deadlift 33333 reps',
 'five rounds for time of  95pound hang squat clean 21 reps run 400 meters',
 'three rounds of 20 walking lunges with dumbbells totaling 13 your bodyweight 20 thrusters 20 pullups rest three minutes  notes 1 measure your pace so that thrusters begin directly under your pullup bar  2 each step of the lun

In [21]:
# To generate an LDA model, we need to understand how frequently each term occurs within each document. 
# To do that, we need to construct a document-term matrix with a package called gensim:

# from gensim import corpora, models
#
texts = [[token_item] for token_item in token_list]

texts

# dictionary = corpora.Dictionary(token_list)

# dictionary

[['seven rounds for time of walking lunge 10 steps 10 pull ups 10 push ups'],
 ['three rounds for time of 1000 meter row 12 bodyweight push jerk 21 reps'],
 ['tabata something else complete 32 intervals of 20 seconds of work followed by ten seconds of rest where the first 8 intervals are pullups the second 8 are pushups the third 8 intervals are situps and finally the last 8 intervals are squats there is no rest between exercises'],
 ['five sets of the triple situp back extension benchpress 510rm move methodically deliberately with precise slow movement  dont hurry minimize the cardiovascular quality this is strength training only  make each effort a serious challenge'],
 ['deadlift 33333 reps'],
 ['five rounds for time of  95pound hang squat clean 21 reps run 400 meters'],
 ['three rounds of 20 walking lunges with dumbbells totaling 13 your bodyweight 20 thrusters 20 pullups rest three minutes  notes 1 measure your pace so that thrusters begin directly under your pullup bar  2 each st

In [19]:
wodBigrams = nltk.bigrams(words)
wodBigrams

<generator object bigrams at 0x11a917b90>

In [20]:
wodTrigrams = nltk.trigrams(words)
wodTrigrams

<generator object trigrams at 0x11a8cbe10>

In [229]:
vectorizer = CountVectorizer(min_df=1)

In [230]:
corpus = wodDict.values()

In [233]:
X = vectorizer.fit_transform(corpus)

/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [234]:
X

<5311x7776 sparse matrix of type '<type 'numpy.int64'>'
	with 100492 stored elements in Compressed Sparse Row format>

In [161]:
soup.prettify

<bound method BeautifulSoup.prettify of <table align="left" border="0" cellpadding="6" cellspacing="2" width="890">\n<tr><td align="left" border="0" cellpadding="0" cellspacing="0" valign="top" width="18">\n</td><td align="left" bgcolor="#cccccc" border="0" cellpadding="0" cellspacing="0" colspan="3" valign="top">\n<a href="http://www.crossfit.com"><img align="left" alt="CrossFit.com" border="0" height="104px" src="http://www.crossfit.com/cf-assets/logos/cf-logo-main.jpg" title="This is The CrossFit.com Main Page" width="876px"/></a>\n</td></tr>\n<tr><td border="0" cellpadding="0" cellspacing="2" valign="top" width="18"><img height="4" src="http://www.crossfit.com/cf-assets/dot_clr.gif" width="18"/></td>\n<td align="left" bgcolor="#FFFFFF" border="0" cellpadding="0" cellspacing="0" valign="top" width="190">\n<table bgcolor="#FFFFFF" border="0" bordercolor="#FFFFFF" cellpadding="0" cellspacing="0" height="100%" valign="top" width="191">\n<tr>\n<td bgcolor="#FFFFFF" valign="top" width="1

In [ ]:
#############################################################
################## MASSAGE ##################################
#############################################################

# Daily Dict
d2 = dict((datetime.strptime(k, '%B %d, %Y'), v) for k, v in wodDict.items())

# Yearly Dict
d3 = {}
for k, v in d2.items():
    if d3.has_key(k.year):
        d3[k.year] += str(v)
    else:
        d3[k.year] = str(v)



#############################################################
################## MOVEMENTS ################################
#############################################################

# from crossfit.com
text_file = open("CFmovements.txt", "r")

Movements = text_file.readlines()
Movements = map(lambda s: s.strip().lower(), Movements)

f = open('CFmovements.txt', 'w') # open for 'w'riting
for item in Movements:
  f.write("%s\n" % item)
f.close() # close the file

# count the number of times a movement occurs in an a string
numDict ={}
for move in Movements:
    numDict[move] = len(re.findall(move,d3[d3.keys()[0]].lower(),flags=re.I))

#for each movement, log the date on which the movement occurs
dateDict ={}
for move in Movements:
    for day in d2:
        if len(re.findall(move,d2[day].lower(),flags=re.I))>0:
            if dateDict.has_key(move):
                dateDict[move].append(day)
            else:
                dateDict[move] = []
                dateDict[move].append(day)

## Monthly Cumm of each Movement
movemonthDict={}
for move in Movements:
    month_aggregate = {}
    if dateDict.has_key(move):
        for d in dateDict[move]:
            month, day, year = d.month, d.day, d.year
            if month_aggregate.has_key(date(year, month, 1)):
                month_aggregate[date(year, month, 1)] += 1
            else:
                month_aggregate[date(year, month, 1)] = 1
        movemonthDict[move]=month_aggregate


pickle.dump(dateDict, open( "dateDict.p", "wb" ) )
dateDict = pickle.load( open( "dateDict.p", "rb" ) )

pickle.dump(movemonthDict, open( "movemonthDict.p", "wb" ) )
movemonthDict = pickle.load( open( "movemonthDict.p", "rb" ) )

#############################################################
################## VISUAL ###################################
#############################################################


################## MATPLOTLIB ###############################

# movemonthDict.keys()
# toplot=movemonthDict['tabata']
# toplot2=movemonthDict['burpee']
# plt.bar(toplot.keys(), toplot.values(), width=7, align='center', color='brown')
# plt.bar(toplot2.keys(), toplot2.values(), width=7, align='center', color='y')
# #plt.gcf().autofmt_xdate()
# plt.show()
########

################## GGPLOT ###################################

# Load Previous Work
movemonthDict = pickle.load( open( "movemonthDict.p", "rb" ) )
wodDict = pickle.load( open( "wodDict.p", "rb" ) )
dateDict = pickle.load( open( "dateDict.p", "rb" ) )

# Subset Dict
keys = ['snatch'] #['tabata', 'burpee', 'snatch', 'front squat', 'rope climb']
mmDictsubset = { k: movemonthDict[k] for k in keys }

# Convert to DataFrame
df = pandas.DataFrame(mmDictsubset)
df['year'] = [d.year for d in df.index]
df['date'] = [d.strftime('%Y%m%d') for d in df.index]
df.sort([str(keys[0]), 'date'], ascending=[1, 1]).head(10)
df_lng = pandas.melt(df, id_vars=['date','year'])
result = df_lng.sort(['variable', 'date'], ascending=[1, 1])


plot = ggplot(df_lng, aes(x='date', y='value'))+\
        geom_bar(aes(x='date', weight = 'value', fill='steelblue'))+\
        ylab('Number of Weekly Workouts w/ Snatch') + xlab('Date')
        # geom_vline(xintercept = 20011001)
        #+ scale_x_date(breaks=date_breaks('1 day'), labels='%b %d %Y')\
        # + scale_y_continuous(labels= 'percent')\
        # scale_x_continuous(breaks=['20011001'])

#print plot

ggsave(plot, "ggplotPythonBar_2.pdf")


#############################################################
################## OFF TO R! ################################
#############################################################

rDF = pd.DataFrame(movemonthDict)
rDF.to_csv('movemonthDict.csv', sep=',')


# Day Dict to R
dateDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dateDict.iteritems() ])) # columns are movements, rows are dates (w/ NaT's)
dateDF.to_csv('dateDF.csv', sep=',')